In [41]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import re

import spacy

In [42]:
ecommerce_data= pd.read_csv("ecommerceDataset.csv", names=['category', 'text'])
ecommerce_data[:5]

,category,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [43]:
ecommerce_data.shape

(50425, 2)

In [44]:
ecommerce_data.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [45]:
ecommerce_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50425 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  50425 non-null  object
 1   text      50424 non-null  object
dtypes: object(2)
memory usage: 788.0+ KB


In [46]:
ecommerce_data["category_label"]= ecommerce_data.category.map({
    "Household": 0,
    "Books": 1,
    "Electronics": 2,
    "Clothing & Accessories": 3
})

In [47]:
ecommerce_data[:5]

,category,text,category_label
0,Household,Paper Plane Design Framed Wall Hanging Motivat...,0
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",0
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...,0
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1...",0
4,Household,Incredible Gifts India Wooden Happy Birthday U...,0


In [48]:
#Checking for missing values
ecommerce_data.isnull().sum()

category          0
text              1
category_label    0
dtype: int64

In [49]:
ecommerce_data[ecommerce_data.text.isnull()]

,category,text,category_label
39330,Clothing & Accessories,NaN,3


In [50]:
#Replacing missing missing value with zero
ecommerce_data["text"]= ecommerce_data.text.fillna(0)

In [51]:
#Dropping row with missing value
ecommerce_data= ecommerce_data[ecommerce_data.text != 0]

In [52]:
ecommerce_data.shape

(50424, 3)

In [53]:
ecommerce_data[:5]

,category,text,category_label
0,Household,Paper Plane Design Framed Wall Hanging Motivat...,0
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",0
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...,0
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1...",0
4,Household,Incredible Gifts India Wooden Happy Birthday U...,0


In [54]:
ecommerce_data.text[4]

'Incredible Gifts India Wooden Happy Birthday Unique Personalized Gift (5 X 4 Inch) Size:4 x 5   Made Of Natural Imported Wood, Which Is Quite Solid With Light Particle Pattern & Is Soft Pale To Blond Colour. Your Uploaded Photo Will Look Amazing And Beautiful After Laser Engraving On It. This Is One Of The Most Popular Unique Gifts In Our Store. We Offer This In Multiple Sizes, Some Can Be Used As Table Top And The Big Sizes Can Be Used As Wall Hanging Which Just Blends With Your Home Decaration. You Just Need To Upload A Picture And Add Your Own Text And We Will Do The Rest For You. We Will Email You The Preview Before Making The Final Product. Do You Want The Best Moment Of Your Life To Be Engraved On A Wooden Plaque That Lasts For A Longer Time And Stays Close To You Forever? Then You Are At The Right Place. We Present To You Various Sizes Personalized Engraved Wooden Plaques Made With Birch Wood. Let Your Memories Be Engraved On Wooden Plaques And Stay With Your Forever.'

In [55]:
nlp= spacy.load("en_core_web_lg")

In [56]:
doc= nlp("Paper Plane Design Framed Wall Hanging Motivation")
doc[0].vector.shape

(300,)

In [57]:
doc[1].vector[:5]

array([ 0.033539,  1.0758  ,  4.2654  ,  2.4083  , -1.4626  ],
      dtype=float32)

In [58]:
doc[1].vector.shape

(300,)

In [59]:
ecommerce_data["vector"]= ecommerce_data["text"].apply(lambda x: nlp(x).vector)

In [60]:
ecommerce_data[:5]

,category,text,category_label,vector
0,Household,Paper Plane Design Framed Wall Hanging Motivat...,0,"[-2.306548, 0.30390584, -2.51085, 0.06632433, ..."
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",0,"[-2.6766863, -1.1272602, -1.1009779, 0.4870543..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...,0,"[-2.414381, 0.7888623, -2.8213925, 0.8304517, ..."
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1...",0,"[-2.3844197, 0.7978983, -2.7950675, 0.74301887..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...,0,"[-0.5195761, -0.46975613, -1.4031031, -0.38848..."


In [116]:
ecommerce_data.vector[:5]

0    [-2.306548, 0.30390584, -2.51085, 0.06632433, ...
1    [-2.6766863, -1.1272602, -1.1009779, 0.4870543...
2    [-2.414381, 0.7888623, -2.8213925, 0.8304517, ...
3    [-2.3844197, 0.7978983, -2.7950675, 0.74301887...
4    [-0.5195761, -0.46975613, -1.4031031, -0.38848...
Name: vector, dtype: object

In [97]:
X = np.array([np.array(x) for x in ecommerce_data.vector.values])
y = ecommerce_data.category_label

In [112]:
X[:5]

array([[-2.306548  ,  0.30390584, -2.51085   , ..., -1.4262657 ,
        -3.3489084 ,  0.57573175],
       [-2.6766863 , -1.1272602 , -1.1009779 , ..., -1.6334178 ,
        -2.4390118 , -0.24040593],
       [-2.414381  ,  0.7888623 , -2.8213925 , ..., -2.1887448 ,
        -3.0587976 ,  0.657897  ],
       [-2.3844197 ,  0.7978983 , -2.7950675 , ..., -2.13979   ,
        -2.9163933 ,  0.48472232],
       [-0.5195761 , -0.46975613, -1.4031031 , ..., -0.5076162 ,
        -2.1032832 , -0.826409  ]], dtype=float32)

In [113]:
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: category_label, dtype: int64

In [99]:
#Applying SMOTE to handle the imbalance dataset

X_sm, y_sm= SMOTE().fit_resample(X, y)

In [100]:
X_sm.shape

(77252, 300)

In [101]:
y_sm.value_counts()

category_label
0    19313
1    19313
3    19313
2    19313
Name: count, dtype: int64

In [102]:
X_train, X_test, y_train, y_test= train_test_split(
    X_sm, 
    y_sm,
    test_size= 0.25,
    random_state= 2022)

In [103]:
y_test.value_counts()

category_label
1    4973
0    4845
2    4813
3    4682
Name: count, dtype: int64

In [104]:
y_train.value_counts()

category_label
3    14631
2    14500
0    14468
1    14340
Name: count, dtype: int64

In [105]:
X_train[:5]

array([[-1.6228391 , -0.8795158 , -0.60475844, ..., -1.4353336 ,
        -1.9335147 , -0.8100008 ],
       [-1.6482172 , -0.17391518, -3.0020814 , ..., -1.6706198 ,
        -2.6095624 ,  0.3783911 ],
       [-2.0876565 , -6.637347  ,  4.3394966 , ..., -4.2294626 ,
         1.3783625 ,  0.32852504],
       [-1.2654543 , -0.1589285 ,  3.9103014 , ..., -2.5920284 ,
        -1.35742   , -2.7579844 ],
       [-1.26067   ,  0.17699975, -2.0038831 , ..., -0.88837624,
        -3.138105  ,  0.7850804 ]], dtype=float32)

In [106]:
X_test[:3]

array([[-2.68139958e+00,  1.10578740e+00, -2.39854264e+00,
         1.62016943e-01,  4.04618359e+00,  2.27395475e-01,
         3.86115700e-01,  3.27266932e+00, -1.20685530e+00,
        -1.91434696e-01,  6.03872108e+00,  7.90966690e-01,
        -3.85298729e+00,  1.03128266e+00,  1.23039854e+00,
         9.48810518e-01,  9.28355157e-01,  3.47808897e-01,
        -8.18302870e-01, -9.25642014e-01,  3.25626791e-01,
        -7.28772938e-01, -2.99995333e-01, -2.94280469e-01,
         1.49256792e-02, -1.14914083e+00, -2.52467346e+00,
        -1.56552231e+00, -8.37992847e-01,  3.54837142e-02,
        -8.51389319e-02, -9.61718321e-01,  3.94072533e-01,
        -1.79745424e+00, -1.30884743e+00,  4.18543845e-01,
         7.88391948e-01,  1.10038745e+00,  1.10155356e+00,
        -2.18544662e-01,  8.22522581e-01, -1.00615472e-01,
        -1.17175245e+00, -5.28661549e-01, -1.34540856e+00,
         1.69308043e+00,  8.42569530e-01, -1.09013748e+00,
        -6.08105838e-01,  5.52668571e-01,  1.15250669e-0

In [107]:
clf= Pipeline([
    ("scaler", MinMaxScaler()),
    ("model", MultinomialNB())
])

clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.70      0.58      0.63      5902
           1       0.80      0.91      0.85      4333
           2       0.64      0.65      0.64      4798
           3       0.76      0.83      0.79      4280

    accuracy                           0.73     19313
   macro avg       0.73      0.74      0.73     19313
weighted avg       0.72      0.73      0.72     19313



In [110]:
clf2= Pipeline([
    ("scaler", MinMaxScaler()),
    ("model", KNeighborsClassifier(n_neighbors=5, metric="euclidean"))
])

clf2.fit(X_train, y_train)
y_pred2= clf2.predict(X_test)
print(classification_report(y_pred2, y_test))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89      4324
           1       0.96      0.97      0.97      4916
           2       0.97      0.91      0.94      5097
           3       0.98      0.93      0.95      4976

    accuracy                           0.94     19313
   macro avg       0.94      0.94      0.94     19313
weighted avg       0.94      0.94      0.94     19313



In [111]:
clf3= Pipeline([
    ("scaler", MinMaxScaler()),
    ("model", RandomForestClassifier())
])

clf3.fit(X_train, y_train)
y_pred3= clf3.predict(X_test)
print(classification_report(y_pred3, y_test))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95      5023
           1       0.97      0.99      0.98      4921
           2       0.96      0.98      0.97      4700
           3       0.98      0.98      0.98      4669

    accuracy                           0.97     19313
   macro avg       0.97      0.97      0.97     19313
weighted avg       0.97      0.97      0.97     19313

